# Domain Generation Algorithm (DGA) Detection

## Authors
 - Gorkem Batmaz (NVIDIA) [gbatmaz@nvidia.com]
 - Bhargav Suryadevara (NVIDIA) [bsuryadevara@nvidia.com]

## Development Notes
* Developed using: RAPIDS v0.12.0 and CLX v0.12
* Last tested using: RAPIDS v0.12.0 and CLX v0.12 on Jan 28, 2020

## Table of Contents
* Introduction
* Data Importing
* Data Preprocessing
* Training and Evaluation
* Inference
* Conclusion

## Introduction
[Domain Generation Algorithms](https://en.wikipedia.org/wiki/Domain_generation_algorithm) (DGAs) are used to generate domain names that can be used by the malware to communicate with the command and control servers. IP addresses and static domain names can be easily blocked, and a DGA provides an easy method to generate a large number of domain names and rotate through them to circumvent traditional block lists. We will use a type of recurrent neural network called the [Gated Recurrent Unit](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21) (GRU) for this example. The [CLX](https://github.com/rapidsai/clx) and [RAPIDS](https://rapids.ai) libraries enable users train their models with up-to-date domain names representative of both benign and DGA generated strings. Using a CLX workflow, this capability could also be used in production. This notebook provides a view into the data science workflow to create a DGA detection implementation.

In [1]:
import os
import time
import cudf
import torch
import shutil
import s3fs
import numpy as np
from datetime import datetime
from sklearn.metrics import accuracy_score, average_precision_score
from clx.analytics.detector_dataset import DetectorDataset
from clx.analytics.dga_detector import DGADetector
from clx.analytics import detector_utils as du
from cuml.preprocessing.model_selection import train_test_split

#### Download Input Dataset from S3

In [2]:
INPUT_CSV = "benign_and_dga_domains.csv"

S3_BASE_PATH = "rapidsai-data/cyber/clx"

In [3]:
# Read Benign and DGA dataset
if not os.path.exists(INPUT_CSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + INPUT_CSV, INPUT_CSV)

#### Load Input Dataset to GPU Dataframe

In [4]:
input_df = cudf.read_csv(INPUT_CSV)

#### Create Train and Test Dataset
We utilize the [`train_test_split` function](https://docs.rapids.ai/api/cuml/0.10/api.html#model-selection-and-data-splitting) from [cuML](https://github.com/rapidsai/cuml) and create a shuffled dataset for training and testing.

In [5]:
domain_train, domain_test, type_train, type_test = train_test_split(input_df, 'type', train_size=0.7)

In [6]:
def create_df(domain_df, type_series):
    df = cudf.DataFrame()
    df['domain'] = domain_df['domain'].reset_index(drop=True)
    df['type'] = type_series.reset_index(drop=True)
    return df

In [7]:
test_df = create_df(domain_test, type_test)
train_df = create_df(domain_train, type_train)

In [8]:
test_df = du.str2ascii(test_df, test_df.shape[0])
train_df = du.str2ascii(train_df, train_df.shape[0])

Because we have only benign and DGA (malicious) categoriesm, the number of domain types need to be set to 2 (`N_DOMAIN_TYPE=2`). Vocabulary size(`CHAR_VOCAB`) is set to 128 ASCII characters. The values below set for `HIDDEN_SIZE`, `N_LAYERS` of the network, and the `LR` (Learning Rate) give an optimum balance for the network size and performance. They might need be set via experiments when working with other datasets.

In [9]:
LR = 0.001
N_LAYERS = 3
CHAR_VOCAB = 128
HIDDEN_SIZE = 100
N_DOMAIN_TYPE = 2

#### Instantiate DGA Detector
Now that the data is ready, the datasets are created, and we've set the parameters for the model, we can use the DGADetector method built into CLX to create and train the model.

In [10]:
dd = DGADetector(lr=LR)
dd.init_model(n_layers=N_LAYERS, char_vocab=CHAR_VOCAB, hidden_size=HIDDEN_SIZE, n_domain_type=N_DOMAIN_TYPE)

#### Create Batches
We need to partition the input dataframe into one or more smaller dataframes per the given batch size for training and testing of a model.

In [11]:
batch_size = 10000
train_dataset = DetectorDataset(train_df, batchsize=batch_size)
test_dataset = DetectorDataset(test_df, batchsize=batch_size)

In [12]:
def create_dir(dir_path):
    print("Verify if directory `%s` is already exists." % (dir_path))
    if not os.path.exists(dir_path):
        print("Directory `%s` does not exists." % (dir_path))
        print("Creating directory `%s` to store trained models." % (dir_path))
        os.makedirs(dir_path)

In [13]:
def cleanup_cache():
    # release memory.
    torch.cuda.empty_cache()

In [14]:
def train_and_eval(dd, train_dataset, test_dataset, epoch, model_dir):
    print("Initiating model training")
    create_dir(model_dir)
    max_accuracy = 0
    prev_model_file_path = ""
    for i in range(1, epoch + 1):
        print("---------")
        print("Epoch: %s" % (i))
        print("---------")
        dd.train_model(train_dataset)
        accuracy = dd.evaluate_model(test_dataset)
        now = datetime.now()
        output_filepath = (
            model_dir
            + "/"
            + "rnn_classifier_{}.pth".format(now.strftime("%Y-%m-%d_%H_%M_%S"))
        )
        if accuracy > max_accuracy:
            dd.save_model(output_filepath)
            max_accuracy = accuracy
            if prev_model_file_path:
                os.remove(prev_model_file_path)
            prev_model_file_path = output_filepath
    print("Model with highest accuracy (%s) is stored to location %s" % (max_accuracy, prev_model_file_path))
    return prev_model_file_path

### Training and Evaluation
Using the function we created above, we now train and evaluate the model.
*NOTE: You may see warnings when you run the training due to a [bug in PyTorch](https://github.com/pytorch/pytorch/issues/27972) which is being actively investigated.*

In [15]:
%%time
epoch = 30
model_dir='/trained_models'
model_filepath = train_and_eval(dd, train_dataset, test_dataset, epoch, model_dir)
cleanup_cache()

Initiating model training
Verify if directory `/trained_models` is already exists.
---------
Epoch: 1
---------


/opt/conda/envs/rapids/lib/python3.7/site-packages/cudf/io/dlpack.py:74: UserWarning: WARNING: cuDF to_dlpack() produces column-major (Fortran order) output. If the output tensor needs to be row major, transpose the output of this function.
  return libdlpack.to_dlpack(gdf_cols)


[100000/1433083 (7%)]	Loss: 6795.11
[200000/1433083 (14%)]	Loss: 5106.35
[300000/1433083 (21%)]	Loss: 3989.68
[400000/1433083 (28%)]	Loss: 3425.64
[500000/1433083 (35%)]	Loss: 3042.67
[600000/1433083 (42%)]	Loss: 2627.99
[700000/1433083 (49%)]	Loss: 2308.45
[800000/1433083 (56%)]	Loss: 2061.49
[900000/1433083 (63%)]	Loss: 2309.10
[1000000/1433083 (70%)]	Loss: 2217.78
[1100000/1433083 (77%)]	Loss: 2256.86
[1200000/1433083 (84%)]	Loss: 2195.49
[1300000/1433083 (91%)]	Loss: 2184.11
[1400000/1433083 (98%)]	Loss: 2206.87
Test set: Accuracy: 383241/614179 (0.6239890976409158)

---------
Epoch: 2
---------
[100000/1433083 (7%)]	Loss: 3119.61
[200000/1433083 (14%)]	Loss: 2323.62
[300000/1433083 (21%)]	Loss: 1738.85
[400000/1433083 (28%)]	Loss: 1550.39
[500000/1433083 (35%)]	Loss: 1369.71
[600000/1433083 (42%)]	Loss: 1180.35
[700000/1433083 (49%)]	Loss: 1039.26
[800000/1433083 (56%)]	Loss: 930.29
[900000/1433083 (63%)]	Loss: 896.43
[1000000/1433083 (70%)]	Loss: 907.86
[1100000/1433083 (77%)]	Lo

### Inference

Using the model generated above, we now score the test dataset against the model to determine if the domain is likely generated by a DGA or benign.

In [16]:
dd = DGADetector()
dd.load_model(model_filepath)

pred_results = []
true_results = []
for partition in test_dataset.get_chunks():
    pred_results.append(list(dd.predict(partition['domain']).values_host))
    true_results.append(list(partition['type'].values_host))
pred_results = np.concatenate(pred_results)
true_results = np.concatenate(true_results)
accuracy_score = accuracy_score(pred_results, true_results)
print('Model accuracy: %s'%(accuracy_score))
cleanup_cache()

Model accuracy: 0.990994482064675


In [17]:
average_precision = average_precision_score(true_results, pred_results)

print('Average precision score: {0:0.3f}'.format(average_precision))

Average precision score: 0.976


## Conclusion

DGA detector in CLX enables users to train their models for detection and also use existing models. This capability could also be used in conjunction with log parsing efforts if the logs contain domain names. DGA detection done with CLX and RAPIDS keeps data in GPU memory, removing unnecessary copy/converts and providing a 4X speed advantage over CPU only implementations. This is esepcially true with large batch sizes.